# Module 1, Activity 1: Getting Started with Bedrock and Basic Prompting

In this notebook we are going to demonstrate how to use the `boto3` Python SDK along with a variety of abstractions available through the LangChain package to work with the Amazon Bedrock Foundational Models.  By the end of this notebook you will be able to create a basic chain with a simple prompt capable of asking questions to the model and outputing the answer in a human-readable format.

In [1]:
import boto3

from langchain_aws import BedrockLLM, ChatBedrock
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate

In [2]:
session = boto3.session.Session()
region = session.region_name

## Create Bedrock management connection

The Bedrock client is used as the control for Bedrock.  It can do things like list models, check availability, and manage configurations.  But it doesn't actually do anything with the models.

In [3]:
bedrock = boto3.client("bedrock", region_name='us-west-2')
model_ids = [model["modelId"] for model in bedrock.list_foundation_models()["modelSummaries"]]
print("Available Models:", model_ids)

Available Models: ['amazon.titan-tg1-large', 'amazon.nova-premier-v1:0:8k', 'amazon.nova-premier-v1:0:20k', 'amazon.nova-premier-v1:0:1000k', 'amazon.nova-premier-v1:0:mm', 'amazon.nova-premier-v1:0', 'amazon.titan-embed-g1-text-02', 'amazon.titan-text-lite-v1:0:4k', 'amazon.titan-text-lite-v1', 'amazon.titan-text-express-v1:0:8k', 'amazon.titan-text-express-v1', 'amazon.nova-pro-v1:0', 'amazon.nova-lite-v1:0', 'amazon.nova-micro-v1:0', 'amazon.titan-embed-text-v1:2:8k', 'amazon.titan-embed-text-v1', 'amazon.titan-embed-text-v2:0', 'amazon.titan-embed-image-v1:0', 'amazon.titan-embed-image-v1', 'amazon.titan-image-generator-v1:0', 'amazon.titan-image-generator-v1', 'amazon.titan-image-generator-v2:0', 'amazon.rerank-v1:0', 'stability.stable-diffusion-xl-v1:0', 'stability.stable-diffusion-xl-v1', 'stability.sd3-large-v1:0', 'stability.sd3-5-large-v1:0', 'stability.stable-image-core-v1:0', 'stability.stable-image-core-v1:1', 'stability.stable-image-ultra-v1:0', 'stability.stable-image-ul

## Initializing BedrockLLM and invoking a model

Here, the BedrockLLM class from the langchain_aws package is instantiated.  This class serves as a high-level wrapper to interface with AWS-hosted LLMs.
The initialization parameters include the model ID (in this case, "amazon.titan-tg1-large"), region, and the necessary AWS credentials.  Once the instance is created, the invoke method is used to send a prompt ("What is the recipe of mayonnaise?") to the model.  This section demonstrates the fundamental workflow: setting up the model wrapper and making a basic invocation call to test the model’s response, providing a concrete example of how to interact with AWS-hosted generative AI models using LangChain.

Try several different prompts here to see what different types of answers you can get!

In [4]:
simple_llm = BedrockLLM(
    model_id="amazon.titan-tg1-large",
    region_name='us-west-2',
)
print(simple_llm.invoke("What is the recipe for mayonnaise?"))


Here is the recipe for mayonnaise:

1. 1 cup of vegetable oil
2. 2 egg yolks
3. 1 tablespoon of lemon juice or vinegar
4. 1 teaspoon of mustard powder
5. 1 teaspoon of salt

1. Whisk together the egg yolks, lemon juice, mustard powder, and salt in a bowl.
2. Gradually add the vegetable oil, whisking continuously until the mixture thickens.
3. Once thickened, transfer the mayonnaise to a jar and refrigerate until ready to use.


## Introducing ChatBedrock

BedrockLLM is designed for single-turn, prompt-based interactions where you provide the prompt ("What is the recipe for mayonnaise?") and the model generates an output in one go.  This is fine for simple things, but when you need to have more sophisticated interactions you want something that supports chat-like exchanges where the model can manage context over several turns of dialogue.  Additionally, not all of the available models, including more sophisticated models like Anthropic's Claude 3 Sonnet below, are supported by BedrockLLM.  Hence, we have the more sophisticated ChatBedrock, as shown below.

Also note that the output of ChatBedrock contains much more information than just a text output.

In [ ]:
chat_llm = ChatBedrock(
    model_id="anthropic.claude-3-sonnet-20240229-v1:0",
    region_name='us-west-2',
)
print(chat_llm.invoke("What is the recipe for mayonnaise?"))

content='Here is a basic recipe for making mayonnaise from scratch:\n\nIngredients:\n- 1 egg yolk\n- 1 tablespoon lemon juice or white wine vinegar\n- 1/2 teaspoon dijon mustard (optional, for flavor)\n- 1/4 teaspoon salt\n- 3/4 cup neutral oil like canola, vegetable or grapeseed oil\n\nInstructions:\n\n1. In a medium bowl, whisk together the egg yolk, lemon juice/vinegar, mustard (if using) and salt.\n\n2. Very slowly, while whisking constantly, drizzle in a few drops of the oil until the mixture begins to thicken and emulsify.\n\n3. Still whisking constantly, start adding the oil in a thin steady stream, going slowly to allow the mixture to emulsify and thicken.\n\n4. Once all the oil has been incorporated and the mayonnaise is thick, you can adjust seasoning if needed by adding more lemon juice, salt, etc.\n\n5. Transfer to an airtight container and refrigerate for up to 5 days.\n\nThe keys are using a room temperature egg yolk, slowly drizzling in the oil while whisking vigorously 

## Temperature

Temperature is the thing that gives models creativity.  It controls the randomness of the model's responses.  Setting it to 0.0 (the minimum) typically results in a more deterministic and consistent output while setting it to 1.0 (the maximum) results in more creative responses.  

- Low temperature (e.g., 0.0-0.3): The model plays it safe, sticking to the most likely answers. This is great when you want accuracy and consistency, like coding help or fact-based answers.

- High temperature (e.g., 0.7-1.0): The model gets more adventurous, picking less common words and generating more diverse responses. This is useful for creative writing, brainstorming, or when you want unique outputs.

If you set it to 0, the model is basically deterministic—it’ll always give the same answer if asked the same thing.  As you increase temperature you will get more creative (and perhaps unpredictable!) responses.  So let's create a simple prompt and run it a few times.  Experiment with both the prompt and the temperature of the following cell and observe the effects.

In [ ]:
chat_llm_temp = ChatBedrock(
    model_id="anthropic.claude-3-sonnet-20240229-v1:0",
    region_name='us-west-2',
    temperature=0.0
)
print(chat_llm_temp.invoke("Write a short bedtime story about a robot."))

content="To generate boilerplate code for a payment processing microservice in Python that integrates with a payment gateway API, we need to consider several key aspects, including security, error handling, and logging. Here's an explanation of how I would approach this task:\n\n1. **Security**:\n   - **Token-based Authentication**: Implement token-based authentication to secure the communication between the microservice and the payment gateway API. This can be achieved using industry-standard protocols like OAuth 2.0 or JSON Web Tokens (JWT). The microservice should obtain an access token from the payment gateway API and include it in the headers of all subsequent requests.\n   - **Encryption**: Ensure that sensitive data, such as credit card numbers and other payment details, are encrypted both in transit and at rest. Use secure protocols like HTTPS for communication and encrypt data stored in databases or files using strong encryption algorithms like AES-256.\n\n2. **Error Handling*

## Limiting the number of tokens returned

The cost of using an LLM is dependent on how many tokens are sent back and forth with the model.  The `max_tokens` parameter can provide a limit on how many total tokens are returned.  Limiting the token count can be useful when you need to ensure that the responses remain concise or when working within strict output size constraints.  Experiment with a few different values for this to see how the output changes.

In [ ]:
chat_llm_limited = ChatBedrock(
    model_id="anthropic.claude-3-sonnet-20240229-v1:0",
    region_name='us-west-2',
    temperature=0.0,
    max_tokens=1000
)
print(chat_llm_limited.invoke("What is the recipe for mayonnaise?"))

content='Here is a basic recipe for homemade mayonnaise:\n\nIngredients:\n- 1 egg yolk\n- 1 tablespoon lemon juice or white wine vinegar\n- 1/2 teaspoon dijon mustard (optional)\n- 1/4 teaspoon salt\n- 3/4 cup vegetable oil or mild olive oil\n\nInstructions:\n\n1. In a medium bowl, whisk together the egg yolk, lemon juice/vinegar, mustard (if using), and salt.\n\n2. Very slowly, while whisking constantly, drizzle in a few drops of the oil until the mixture begins to thicken. \n\n3. Once thickened, you can add the oil in a thin steady stream while continuing to whisk vigorously. \n\n4. Whisk until all the oil is incorporated and the mayonnaise is thick and emulsified.\n\n5. Taste and adjust seasoning if needed by adding more lemon juice, salt, etc.\n\n6. Transfer to an airtight container and refrigerate for up to 1 week.\n\nThe key is adding the oil very slowly while whisking constantly to allow a stable emulsion to form between the egg yolk and oil. Be patient and go slowly when adding

## Creating a Chain with Prompts and Output Parsing

Now that we have seen some of the basics, it is time to create an actual question-answering bot.  In order to turn this into a fully-functioning question-answering system, we need to combine the following three concepts:

### Prompts and Prompt Templates

In LangChain, **system prompts** (provide the foundational instructions and context for the AI model.  They define the model's role, behavior, and any specific output format or constraints, ensuring that all subsequent responses align with the intended purpose.  In contrast, **human prompts** represent the dynamic, user-supplied inputs that drive the conversation or query.  They capture the specific question or instruction that the user wants the model to address.  The `ChatPromptTemplate` acts as a framework that seamlessly integrates both types of prompts into a coherent message sequence.  By combining system and human messages, the `ChatPromptTemplate` creates a structured dialogue that guides the model to generate responses that are both contextually relevant and properly formatted.

### Chains

Chains in LangChain are modular sequences that connect various components—such as prompt templates, language models, and output parsers—into a unified workflow.  They enable developers to construct complex, multi-step processing pipelines where each step transforms or utilizes the output from the previous one.  LCEL (LangChain Chain Expression Language) implements this concept using a concise, pipe operator (|) syntax that clearly defines the data flow between these components.  By leveraging LCEL, you can declaratively compose chains that are both flexible and readable, allowing for rapid prototyping and iterative development of sophisticated AI-powered applications.

### Output Parsers

The `AIMessage` format might not be the best way for the output of your LLM call to be rendered.  So we can keep adding to the chain by adding another component: `StrOutputParser`.  It converts a language model's output into a clean, plain text string by stripping away any additional metadata or formatting.  So we can see in this example that we just add it modularly to the end of our chain.  There are several other output parsers that you might find helpful.  You can find a listing of them in [this LangChain documentation](https://python.langchain.com/docs/how_to/#output-parsers). 

In this code block we are creating a simple, three-part chain with our prompt, our LLM, and our output parser.

And be sure to check out the system prompt to understand the response you are about to get!  We are not going to tinker too much with prompts just yet.  That will be the subject of the next activity.  Just for now know that you can add some basic instructions here to tell the model what you are looking for.


In [21]:
system_prompt = SystemMessagePromptTemplate.from_template(
    "You are a helpful assistant."
)
human_prompt = HumanMessagePromptTemplate.from_template("{input}")
prompt = ChatPromptTemplate.from_messages([system_prompt, human_prompt])

# Chain with prompt and LLM
chain = prompt | chat_llm_temp | StrOutputParser()
#print(chain.invoke({""}))
print(chain.invoke({"Create a YAML linter to generate a YAML config with three Python linter rules for a codebase.Each rule must include:rule_name: short, descriptive stringseverity: "warning" or "error"description: brief explanationThe rules you must generate are:no-hardcoded-credentials — flag hardcoded secrets like API keys or DB passwordsvalidate-currency-format — ensure currency values are consistently formatteduse-typed-annotations — encourage use of Python type hints for clarityUse a custom output parser that:Strips markdown/code fencesParses the YAML with yaml.safe_load()Validates that rules is a list of 3 dicts, each with the required keysYour final output must be valid YAML only — no markdown, no extra text.Example:rules:  - rule_name: no-hardcoded-credentials    severity: error    description: Prevent hardcoding of sensitive credentials like API keys.  ...
"}))

SyntaxError: invalid syntax. Perhaps you forgot a comma? (1697979344.py, line 10)